In [1]:
#!/usr/bin/python
# Modified 11/29/2019 by HYK
# Incorporate binomial partitioning, DONE!
# Cumulative addition of time over multiple rounds of simulation, DONE!
# activation cascade with three components
# Generalized functions for simulating arbitrary reaction network topologies
# Simulation cell number distributions as well, for multiple runs
# Also, write a MATLAB script to rapidly generate plots from the simulated data

"""
Created on Tue Dec 13 20:08:54 2016
@author: Sam Nguyen
"""
import roadrunner
import numpy as np
from numpy import random as rand
import math
from scipy import asarray as ar,exp
from multiprocessing import Pool
import csv
import random
import sys
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
roadrunner.Config.setValue(roadrunner.Config.MAX_OUTPUT_ROWS,10000)

# Tellurium loader
def loada(sbstr):
    import antimony as sb
    r = sb.loadAntimonyString(sbstr)
    if r < 0:
        raise RuntimeError('Failed to load Antimony model: {}'.format(sb.getLastError()))
    return roadrunner.RoadRunner(sb.getSBMLString(sb.getModuleNames()[-1]))

# Tellurium simulation function
def doSimulation(tc,nc,gamma_x,gamma_y):
    r = loada("""
    # define cell cycling parameters
    TCycle = 10                                    
    V := (time/TCycle + 1)
    
    #define ode    
    R1: => Px ; (kX*V)*(S/V)*(1-Px)*(KY/V)/(KY/V+Y/V)      # promoter activation (irreversible)
    R2: => X ; (alphaX*V)*Px           # synthesis of X, rate increases with 
    R3: X => ; X*deltaX                # degradation of X
    R4: => Py ; (kY*V)*(S/V)*(1-Py)*(KX/V)/(KX/V+X/V)     # activation of pY by X
    R5: => Y ; (alphaY*V)*Py
    R6: Y => ; Y*deltaY

   
   #define initial values
    S = 1            # the concentration of signal * (normalized units) 
    Px = 0             # initial promoter state of X in the state (inactive)  
    X = 0              # the number of copies of X
    Py = 0              # the promoter state of Y
    Y = 0              # the number of copies of Y

    kX = 0.02
    KX = 20
    alphaX = 1300          # synthesis rate of X (copies/hr)
    deltaX = 1             # degradation rate (1/hr)
    kY = 0.01
    KY = 20
    alphaY = 1000         # synthesis rate of Y (copies/hr)
    deltaY = 1             # degradation rate (1/hr)
    """)
    # set parameters of stochastic simulation
    r.integrator = 'gillespie'
    r.integrator.variable_step_size = False  
    r.TCycle = tc # define cell cycle time 
    
    # redefine parameters here, if necessary
    r.alphaX = gamma_x
    r.alphaY = gamma_y
    ############################
    dt = 0.1;     # time interval for plotting
    result=[]
    for i in range(0,nc,1):       # for loop over the total number of cell cycles
        if i == 0:
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','Px','X','Py','Y']) 
            for row in SubResult:
                result.append(row)
        else:
            r.X = rand.binomial(r.X, 0.5)   # randomly partition X due to cell division
            r.Y = rand.binomial(r.Y, 0.5)   # randomly partition Y due to cell division
            SubResult =r.simulate(0,tc,int(tc/dt),['time','V','Px','X','Py','Y'])   # automatic update of activation state?            
            for row in SubResult:
                row[0]=row[0]+(i*tc)     # update the time vector
                result.append(row)
    result = ar(result)
    return result

#Write csv file
def writeCSV(filename,result):
    with open(filename,'w') as mycvsfile:
        thedatawriter=csv.writer(mycvsfile)   
        thedatawriter.writerows(result)

        
#generate fractional overlap
def generate_fraction(repnum,gamma_x,gamma_y,thresh):
    
    Xonly =0
    Yonly =0
    both = 0
    none = 0
    for rep in range(repnum):
        #print('rep: {}'.format(rep))
        df = pd.read_csv('/external/Sam/JRSI/MutualInhibition/data/data_{}_gx_{}_gy_{}.csv'.format(rep,np.round(gamma_x,2),np.round(gamma_y,2)),header = None)
        X = df.iloc[:,2]
        Y = df.iloc[:,4]
        x_ind = X[X>thresh].index
        y_ind = Y[Y>thresh].index
        
        if len(x_ind) == 0 and len(y_ind) == 0:
            none += 1
        elif len(x_ind) != 0 and len(y_ind) != 0:
            both += 1
        elif len(x_ind) == 0 and len(y_ind) != 0:
            Yonly += 1
        else:
            Xonly += 1
    
    return float(Xonly)/repnum,float(Yonly)/repnum,float(both)/repnum,float(none)/repnum
            
#run a series of simulation replicates for a fixed parameter set
def runsimset(repnum,gamma_x,gamma_y):
    for i in range(0,repnum,1):
        result = doSimulation(10,40,gamma_x,gamma_y)
        #file format: t V PX X PY Y
        writeCSV('/external/Sam/JRSI/MutualInhibition/data/data_{}_gx_{}_gy_{}.csv'.format(i,np.round(gamma_x,2),np.round(float(gamma_y),2)),result)


In [ ]:
#### MAIN SCRIPT #######
print(' run the simulation over a range of gamma_x and gamma_y parameters...')
gx_range = np.arange(2200.0,2400.0,200.0)
gy_range = np.arange(2200.0,2400.0,200.0)

for gx in gx_range:
    for gy in gy_range:
        runsimset(1000,float(gx),float(gy))

In [ ]:
gx_range = np.arange(200.0,2400.0,200.0)
gy_range = np.arange(200.0,2400.0,200.0)
thresh = 0
print(' populate the fraction of Xon, Yon in matrices with row = gx_range and col = gy_range...')

XON = np.zeros([len(gx_range),len(gy_range)])
YON = np.zeros([len(gx_range),len(gy_range)])
XYON = np.zeros([len(gx_range),len(gy_range)])
XYOFF= np.zeros([len(gx_range),len(gy_range)])
for gx_ind in range(len(gx_range)):
    for gy_ind in range(len(gy_range)):
        print('gx: {}'.format(gx_range[gx_ind]))
        print('gy: {}'.format(gy_range[gy_ind]))
        XON[gx_ind,gy_ind],YON[gx_ind,gy_ind],XYON[gx_ind,gy_ind],XYOFF[gx_ind,gy_ind] =\
        generate_fraction(1000,gx_range[gx_ind],gy_range[gy_ind],thresh)
        
        


In [ ]:
gx_range = [str(x) for x in gx_range]
gy_range = [str(x) for x in gy_range]

print(' plot this as a heatmap..')
XON_df = pd.DataFrame(XON,columns = gx_range,index = gy_range)
YON_df = pd.DataFrame(YON,columns = gx_range,index = gy_range)
XYON_df = pd.DataFrame(XYON,columns = gx_range,index = gy_range)
XYOFF_df = pd.DataFrame(XYOFF,columns = gx_range,index = gy_range)

#ax = sns.heatmap(XON_df,square = True,vmax = 1,vmin = 0)
#ax.invert_yaxis()
#plt.savefig('testX.svg')
ax = sns.heatmap(YON_df,square = True,vmax = 1,vmin = 0)
ax.invert_yaxis()
#plt.show()
plt.savefig('testY.svg')
# ax = sns.heatmap(XYON_df,square = True,vmax = 0.3,vmin = 0)
# ax.invert_yaxis()
# plt.savfig('testXY.svg')
# plt.show()
# ax = sns.heatmap(XYOFF_df,square = True,vmax = 1,vmin = 0)
# ax.invert_yaxis()
# plt.show()